In [3]:
import json
import pickle as pkl
import re
from nltk.translate import bleu_score
from tqdm import tqdm
import jiwer
from matplotlib import pyplot as plt
import copy
import os

In [4]:
# Define sentence transformation
transformation = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemoveMultipleSpaces(),
    jiwer.ExpandCommonEnglishContractions(),
    jiwer.RemovePunctuation(),
    jiwer.Strip()
])

In [21]:
# Load Friends Data
dir_path = 'emory_nlp_dataset/json/'
output = {}
for file_name in os.listdir(dir_path):
    print(file_name)
    # Load input data
    input_path = dir_path + file_name
    output_path = 'emory_nlp_dataset/pkl/friends_' + file_name.strip().split('_')[-1].split('.')[0] + '.pkl'
    with open(input_path) as f:
        data = json.load(f)
    episodes = data['episodes']
    print(type(episodes), len(episodes))
    temp = episodes[0]
    for episode in episodes:
        transcripts = []
        episode_id = episode['episode_id']
        print(episode_id)
        scenes = episode['scenes']
        for scene in scenes:
            utterances = scene['utterances']
            for utt in utterances:
                transcript = utt['transcript'].strip()
                utt_id = utt['utterance_id']
                transcripts.append([transcript, utt_id])
        output[episode_id] = transcripts
data = output

friends_season_01.json
<class 'list'> 24
s01_e01
s01_e02
s01_e03
s01_e04
s01_e05
s01_e06
s01_e07
s01_e08
s01_e09
s01_e10
s01_e11
s01_e12
s01_e13
s01_e14
s01_e15
s01_e16
s01_e17
s01_e18
s01_e19
s01_e20
s01_e21
s01_e22
s01_e23
s01_e24
friends_season_06.json
<class 'list'> 25
s06_e01
s06_e02
s06_e03
s06_e04
s06_e05
s06_e06
s06_e07
s06_e08
s06_e09
s06_e10
s06_e11
s06_e12
s06_e13
s06_e14
s06_e15
s06_e16
s06_e17
s06_e18
s06_e19
s06_e20
s06_e21
s06_e22
s06_e23
s06_e24
s06_e25
friends_season_10.json
<class 'list'> 18
s10_e01
s10_e02
s10_e03
s10_e04
s10_e05
s10_e06
s10_e07
s10_e08
s10_e09
s10_e10
s10_e11
s10_e12
s10_e13
s10_e14
s10_e15
s10_e16
s10_e17
s10_e18
friends_season_07.json
<class 'list'> 24
s07_e01
s07_e02
s07_e03
s07_e04
s07_e05
s07_e06
s07_e07
s07_e08
s07_e09
s07_e10
s07_e11
s07_e12
s07_e13
s07_e14
s07_e15
s07_e16
s07_e17
s07_e18
s07_e19
s07_e20
s07_e21
s07_e22
s07_e23
s07_e24
friends_season_04.json
<class 'list'> 24
s04_e01
s04_e02
s04_e03
s04_e04
s04_e05
s04_e06
s04_e07
s04_e08
s04

In [24]:
for x in list(data.keys())[:2]:
    print(x)
    print(data[x])

s01_e01
[["There's nothing to tell! He's just some guy I work with!", 's01_e01_c01_u001'], ["C'mon, you're going out with the guy! There's gotta be something wrong with him!", 's01_e01_c01_u002'], ['All right Joey, be nice. So does he have a hump? A hump and a hairpiece?', 's01_e01_c01_u003'], ['Wait, does he eat chalk?', 's01_e01_c01_u004'], ['', 's01_e01_c01_u005'], ["Just, 'cause, I don't want her to go through what I went through with Carl- oh!", 's01_e01_c01_u006'], ["Okay, everybody relax. This is not even a date. It's just two people going out to dinner and- not having sex.", 's01_e01_c01_u007'], ['Sounds like a date to me.', 's01_e01_c01_u008'], ['', 's01_e01_c01_u009'], ["Alright, so I'm back in high school, I'm standing in the middle of the cafeteria, and I realize I am totally naked.", 's01_e01_c01_u010'], ['Oh, yeah. Had that dream.', 's01_e01_c01_u011'], ["Then I look down, and I realize there's a phone... there.", 's01_e01_c01_u012'], ['Instead of...?', 's01_e01_c01_u013'

In [14]:
key_list = list(data.keys())

In [15]:
for item in key_list[:2]:
    print(item)
    temp = data[item]
    for x in temp:
        print(x)
    print('=='*60)

s01_e01
["There's nothing to tell! He's just some guy I work with!", 's01_e01_c01_u001']
["C'mon, you're going out with the guy! There's gotta be something wrong with him!", 's01_e01_c01_u002']
['All right Joey, be nice. So does he have a hump? A hump and a hairpiece?', 's01_e01_c01_u003']
['Wait, does he eat chalk?', 's01_e01_c01_u004']
['', 's01_e01_c01_u005']
["Just, 'cause, I don't want her to go through what I went through with Carl- oh!", 's01_e01_c01_u006']
["Okay, everybody relax. This is not even a date. It's just two people going out to dinner and- not having sex.", 's01_e01_c01_u007']
['Sounds like a date to me.', 's01_e01_c01_u008']
['', 's01_e01_c01_u009']
["Alright, so I'm back in high school, I'm standing in the middle of the cafeteria, and I realize I am totally naked.", 's01_e01_c01_u010']
['Oh, yeah. Had that dream.', 's01_e01_c01_u011']
["Then I look down, and I realize there's a phone... there.", 's01_e01_c01_u012']
['Instead of...?', 's01_e01_c01_u013']
["That's ri

In [31]:
# Load Memor Dataset
with open('memor/data.json') as f:
    source = json.load(f)

In [32]:
data = {}
for x in tqdm(source):
    instance = copy.deepcopy(source[x])
    sentences = []
    for sentence in source[x]['sentences']:
        sentences.append(transformation(sentence))
    # print(instance)
    instance['sentences_transformed'] = sentences
    data[x] = instance

100%|██████████| 5502/5502 [00:02<00:00, 1841.87it/s]


In [33]:
pkl.dump(data, open('memor/data_transformed.pkl', 'wb'))

In [20]:
with open('episode_indexs_former_10.pkl', 'rb') as f:
    data = pkl.load(f)
for x in data:
    print(x)
    temp = data[x]
    for item in temp:
        print(item)

s01_e01
[3590843, "There's gotta be something wrong with", "There's gotta be something wrong with him.", '他一定有什么毛病.']
[1664584, 'So does he have a hump', 'So does he have a hump and a hairpiece?', '他驼背？ 既驼背又带假发？']
[451073, "I don't want her to go", "I don't want her to go to a zoo. I want to set her free.", '我不想让她去动物园,我想让她自由']
[944537, "I don't want her to go", "I don't want her to go away.", '我不会和她分开的 听清楚了吗？ 医生']
[1664586, "I don't want her to go", "I don't want her to go through what I did with Carl.", '我只是不想你重蹈我和卡尔的覆辙']
[1854679, "I don't want her to go", "I don't want her to go.", '我觉得\ue4d1不想让她走']
[3490867, "I don't want her to go", "I don't even want to go. I don't want her to go.", '我什至都不想去 我不想让她去']
[3964609, "I don't want her to go", "I don't want her to go on.", '我不想让她再继续了']
[1664587, 'This is not even a date', 'Okay, everybody relax. This is not even a date.', '各位别急， 这不算约会']
[1664588, "It's just two people going out", "It's just two people going out to dinner and not having s

In [33]:
# Load Friends Data
dir_path = 'emory_nlp_dataset/json/'
output = {}
for file_name in os.listdir(dir_path):
    # print(file_name)
    # Load input data
    input_path = dir_path + file_name
    output_path = 'emory_nlp_dataset/pkl/friends_' + file_name.strip().split('_')[-1].split('.')[0] + '.pkl'
    with open(input_path) as f:
        data = json.load(f)
    episodes = data['episodes']
    # print(type(episodes), len(episodes))
    temp = episodes[0]
    for episode in episodes:
        transcripts = []
        episode_id = episode['episode_id']
        # print(episode_id)
        scenes = episode['scenes']
        for scene in scenes:
            utterances = scene['utterances']
            for utt in utterances:
                transcript = utt['transcript'].strip()
                utt_id = utt['utterance_id']
                transcripts.append([transcript, transformation(transcript), utt_id])
        output[episode_id] = transcripts

In [35]:
with open('friends_transformed.pkl', 'wb') as f:
    pkl.dump(output, f)

In [34]:
for x in list(output.keys())[:2]:
    temp = {}
    for instance in output[x]:
        print(instance)
    pass

["There's nothing to tell! He's just some guy I work with!", 'there is nothing to tell he is just some guy i work with', 's01_e01_c01_u001']
["C'mon, you're going out with the guy! There's gotta be something wrong with him!", 'c amon you are going out with the guy there is gotta be something wrong with him', 's01_e01_c01_u002']
['All right Joey, be nice. So does he have a hump? A hump and a hairpiece?', 'all right joey be nice so does he have a hump a hump and a hairpiece', 's01_e01_c01_u003']
['Wait, does he eat chalk?', 'wait does he eat chalk', 's01_e01_c01_u004']
['', '', 's01_e01_c01_u005']
["Just, 'cause, I don't want her to go through what I went through with Carl- oh!", 'just cause i do not want her to go through what i went through with carl oh', 's01_e01_c01_u006']
["Okay, everybody relax. This is not even a date. It's just two people going out to dinner and- not having sex.", 'okay everybody relax this is not even a date it is just two people going out to dinner and not havi

In [36]:
with open('friends_transformed.pkl', 'rb') as f:
    data = pkl.load(f)

In [38]:
for x in list(data.keys())[:2]:
    print(x)
    print(data[x])

s01_e01
[["There's nothing to tell! He's just some guy I work with!", 'there is nothing to tell he is just some guy i work with', 's01_e01_c01_u001'], ["C'mon, you're going out with the guy! There's gotta be something wrong with him!", 'c amon you are going out with the guy there is gotta be something wrong with him', 's01_e01_c01_u002'], ['All right Joey, be nice. So does he have a hump? A hump and a hairpiece?', 'all right joey be nice so does he have a hump a hump and a hairpiece', 's01_e01_c01_u003'], ['Wait, does he eat chalk?', 'wait does he eat chalk', 's01_e01_c01_u004'], ['', '', 's01_e01_c01_u005'], ["Just, 'cause, I don't want her to go through what I went through with Carl- oh!", 'just cause i do not want her to go through what i went through with carl oh', 's01_e01_c01_u006'], ["Okay, everybody relax. This is not even a date. It's just two people going out to dinner and- not having sex.", 'okay everybody relax this is not even a date it is just two people going out to dinn